In [12]:
import pandas as pd
import numpy as np
import os
import json
import pydicom
from tqdm import tqdm

In [2]:
dirs = ['cac_0','cac_1']

In [10]:
cac_score_df = pd.read_csv('HRV10_CAC.csv')[['patid','dov','cac']]
cac_score_df = cac_score_df[~np.isnan(cac_score_df['cac'])]

/home/jg/miniconda3/envs/cac/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [62]:
def match_date_format(x):
    return x.replace('-','')
cac_score_df['dov'] = cac_score_df['dov'].map(match_date_format)

In [59]:
# pat id check
values = cac_score_df['patid'].values
d = {k:True for k in cac_score_df['patid'].values}


for dir_name in dirs:
    items = os.listdir(dir_name)
    success = []
    failure = []
    for item in tqdm(items):
        dcm_path = os.path.join(dir_name, item)
        dcm = pydicom.read_file(dcm_path)
        if d.get(int(dcm.PatientID),None):
            success.append(dcm_path)
        else:
            failure.append(dcm_path)
    print(dir_name, len(success), len(failure))


  0%|          | 1/968 [00:00<01:53,  8.50it/s]

832 680


100%|██████████| 968/968 [01:49<00:00,  8.84it/s]

368 600


In [72]:
def getScore(dcm_path , score_df):
    dcm = pydicom.read_file(dcm_path)
    study_date = dcm.StudyDate
    pat_id = int(dcm.PatientID)
    pat_df = score_df[score_df['patid']== pat_id]
    if len(pat_df)>0:
        df = pat_df[pat_df['dov'] == study_date]
        if len(df)>0:
            return [pat_id, study_date, df['cac'].values[0]]
        else:
            return False
    else:
        return False

In [73]:
matched_files_list = []
for dir_name in dirs:
    items = os.listdir(dir_name)
    for item in tqdm(items):
        dcm_path = os.path.join(dir_name, item)
        values = getScore(dcm_path, cac_score_df)
        if values:
            values = [dcm_path] + values
            # values: dcm_path, pat_id, study_date, cac score
            matched_files_list.append(values)

100%|██████████| 968/968 [01:17<00:00, 12.45it/s] 


In [76]:
df = pd.DataFrame(matched_files_list)

In [78]:
df.columns = ['dcm_path', 'pat_id','study_date','score']

In [79]:
df.to_parquet('training.parquet', engine='pyarrow')

In [81]:
dcm.pixel_array

array([[ 96,  96,  96, ..., 125, 128, 131],
       [ 96,  96,  96, ..., 125, 128, 131],
       [ 96,  96,  96, ..., 124, 128, 132],
       ...,
       [364, 457, 522, ..., 478, 477, 465],
       [410, 504, 569, ..., 473, 478, 465],
       [456, 550, 616, ..., 453, 462, 450]], dtype=uint16)